In [1]:
# mindspore
import sys
from pathlib import Path
current_dir = Path.cwd()
project_dir = current_dir.parent.parent
sys.path.append(str(project_dir))

from apss.nets.attention_model import AttentionModel
from apss.nets.attention_model_v2 import AttentionStepModel


In [2]:
import mindspore as ms
from apss.utils import load_problem
import numpy as np

def set_decode_type(model, decode_type):
    # if isinstance(model, data_parallel):
    # model只是实例化了一个Cell类，没有调用Model类进行封装
    # model = model._network
    model.set_decode_type(decode_type)

np.random.seed(0)

problem = load_problem("pp")

model_class = {
    'attention': AttentionModel,
    'attention_v2': AttentionStepModel,
}.get("attention", None)
assert model_class is not None, "Unknown model: {}".format(model_class)
model = model_class(
    128,
    128,
    problem,
    n_encode_layers=3,
    mask_inner=True,
    mask_logits=True,
    normalization="batch",
    tanh_clipping=10.,
    # checkpoint_encoder=opts.checkpoint_encoder,
    # shrink_size=opts.shrink_size,
    num_split=3,
    node_size=30
)

param_dict = ms.load_checkpoint("/root/APSS/checkpoint/pp_30_4/pp_30_4_final.ckpt")
ms.load_param_into_net(model, param_dict)

# split_x = ms.Tensor(np.random.rand(4, 29, 5).astype(np.float32))
# ori_x = ms.Tensor(np.random.rand(4, 30, 1).astype(np.float32))
# cost_c_x = ms.Tensor(np.random.rand(4, 29, 3).astype(np.float32))
# np.save('split_x.npy', split_x.asnumpy())
# np.save('ori_x.npy', ori_x.asnumpy())
# np.save('cost_c_x.npy', cost_c_x.asnumpy())

split_x_np = np.load('split_x.npy')
ori_x_np = np.load('ori_x.npy')
cost_c_x_np = np.load('cost_c_x.npy')
split_x = ms.Tensor(split_x_np)
ori_x = ms.Tensor(ori_x_np)
cost_c_x = ms.Tensor(cost_c_x_np)

set_decode_type(model, "greedy")
model.set_train(False)

cost, log_likelihood, pi = model(split_x, ori_x, cost_c_x, return_pi=True)
print(cost)
print(pi)


[12.549763   7.042908   6.547665   5.5664344]
[[24 17 25]
 [ 9 20 19]
 [11 20 21]
 [25  1 16]]


In [ ]:
# pytorch

from nets.attention_model import AttentionModel
from nets.attention_model_v2 import AttentionStepModel

In [ ]:
import torch
from utils import load_problem
import numpy as np

def set_decode_type(model, decode_type):
    # if isinstance(model, data_parallel):
    # model只是实例化了一个Cell类，没有调用Model类进行封装
    # model = model._network
    model.set_decode_type(decode_type)

torch.manual_seed(1234)

problem = load_problem("pp")

model_class = {
    'attention': AttentionModel,
    'attention_v2': AttentionStepModel,
}.get("attention", None)
assert model_class is not None, "Unknown model: {}".format(model_class)
model = model_class(
    128,
    128,
    problem,
    n_encode_layers=3,
    mask_inner=True,
    mask_logits=True,
    normalization="batch",
    tanh_clipping=10.,
    # checkpoint_encoder=opts.checkpoint_encoder,
    # shrink_size=opts.shrink_size,
    num_split=3,
    node_size=30
)

param_dict = torch.load("/home/oj/distributed_floder/data01-oj/oujie/crlsf-pp/outputs/pp_30/pp30_4_rollout_20230327T000146/epoch-99.pt")
param_dict = param_dict["model"]
model.load_state_dict(param_dict)


# split_x = torch.Tensor(np.ones((4 ,29, 5), dtype=np.float32))
# ori_x = torch.Tensor(np.ones((4 ,30, 1), dtype=np.float32))
# cost_c_x = torch.Tensor(np.ones((4 ,29, 3), dtype=np.float32))
split_x_np = np.load('split_x.npy')
ori_x_np = np.load('ori_x.npy')
cost_c_x_np = np.load('cost_c_x.npy')
split_x = torch.from_numpy(split_x_np)
ori_x = torch.from_numpy(ori_x_np)
cost_c_x = torch.from_numpy(cost_c_x_np)

set_decode_type(model, "greedy")
model.eval()

cost, log_likelihood, pi = model(split_x, ori_x, cost_c_x, return_pi=True)
print(cost)
print(pi)